## Job creation on Figure Eight

In the following, we will create a video summary validation task, with CML for the main task using the FigureEight API.

### 1. Preparations

In [1]:
import requests
import json
import csv
import datetime as dt
import pandas as pd
import numpy as np
import re 
from datetime import datetime
import time
import os
from collections import OrderedDict
from gensim.summarization.summarizer import summarize
import itertools 

C:\Users\LTS WinPro\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 2. Time conversion functions

In [2]:
def convert_time(timestring):

    nums = list(map(float, re.findall(r'\d+', timestring)))
    return 3600*nums[0] + 60*nums[1] + nums[2] + nums[3]/1000

In [3]:
def to_seconds(string):
    
    hr, min, sec = [float(x) for x in string.split(':')]
    return hr*3600 + min*60 + sec

### 3. Input preparation

In [4]:
def count_row(file):
    
    df =pd.read_csv(file)
    return df.shape[0]

In [5]:
def starting_time(file, count):
    
    
    df =pd.read_csv(file)
    time = df['starting_time'][count]
    st_time = to_seconds(time)
    
    return st_time

In [6]:
def ending_time(file, count):
    
    
    df =pd.read_csv(file)
    time = df['ending_time'][count]
    end_time = to_seconds(time)
    
    return end_time

In [17]:
substring = 'stoi.srt' #name of the srt file reference
substring_machine = 'myth_machine.txt'

In [18]:
for root, subdirs, files in os.walk('C:/Users/LTS WinPro/Desktop/'): #Your filepath
    for filename in files:
        if substring in filename:
            f = open(os.path.join(root,filename))
            lines = f.readlines()

In [19]:
for root, subdirs, files in os.walk('C:/Users/LTS WinPro/Desktop/'): #Your filepath
    for filename in files:
        if substring_machine in filename:
            f = open(os.path.join(root,filename))
            lines_machine = f.readlines()
            lines_machine = ''.join(lines_machine)
            # print(lines_machine)
            lines_cleaned = lines_machine.rstrip("\n")

In [20]:
times_texts = []
current_times , current_text = None, ""
for line in lines:
    times = re.findall("[0-9]*:[0-9]*:[0-9]*,[0-9]*", line)
    if times != []:
        current_times = list(map(convert_time, times))
    elif line == '\n':
        times_texts.append((current_times, current_text))
        current_times, current_text = None, ""
    elif current_times is not None:
        current_text = current_text + line.replace("\n"," ")

In [21]:
def find_continuous_intervals(inp_time, out_time):
    lists = []
    for y in range(int(inp_time),int(out_time+1)):
        lists.append(y)
    return lists

In [22]:
def find_overlap_intervals(lists):
    return list(set.intersection(*map(set, lists)))

In [23]:
def flat_intervals(intervals):
    lists = []
    for sublist in intervals:
        for item in sublist:
            lists.append(item)
    return lists

In [24]:
def frequent_intervals(intervals):
    lists = []
    num_counter = {}
    for number in intervals:
        if number in num_counter:
            num_counter[number] += 1
        else:
            num_counter[number] = 1
 
    lists = sorted(num_counter, key = num_counter.get, reverse = True)
    top_3 = lists[:300]
    top_3.sort()
    return top_3

In [25]:
def extract_intervals(top_list): 
      
    top_list = sorted(set(top_list)) 
    for key, group in itertools.groupby(enumerate(top_list), 
    lambda t: t[1] - t[0]): 
        group = list(group) 
        yield [group[0][1], group[-1][1]] 

### 4. Main function to produce value structure

In [26]:
res = []
file_csv = 'C:/Users/LTS WinPro/Desktop/stoicism.csv'

counter = count_row(file_csv)
continuousIntervals = []
flatIntervals = []
aggregatedIntervals = []
print(counter)

for i in range(counter):
    inp_time = float(starting_time(file_csv, i))
#    print(inp_time)
    out_time = float(ending_time(file_csv, i))
#    print(out_time)
    intervals = find_continuous_intervals(inp_time, out_time)
    continuousIntervals.append(intervals)
        
flatIntervals = flat_intervals(continuousIntervals)
topTime = frequent_intervals(flatIntervals)
#print(topTime)
aggregatedIntervals = list(extract_intervals(topTime))
#print(aggregatedIntervals)

for i in aggregatedIntervals:
    for key,value in times_texts:
        if ((key[0] > i[0]) and (key[1] < i[1])):
            #print(value)
            temp = [key,value]
            res.append(temp)
            #res.append(key,value)
#print(res)

s=""
str=""
for key, value in res:
    #print(value)
    sst = s.join(value)
    str+=sst

print(str)

17
You've been stranded thousands of miles from home with no money or possessions. Such a predicament would make many people despair and curse their awful fate. But for Zeno of Cyprus, it became the foundation of his life's work and legacy. The once wealthy merchant lost everything when he was shipwrecked in Athens around 300 BCE. With not much else to do, he wandered into a book shop, became intrigued by reading about Socrates, and proceeded to seek out and study with the city's noted philosophers. As Zeno began educating his own students, he originated the philosophy known as Stoicism, whose teachings of virtue, tolerance, and self-control have inspired generations of thinkers and leaders. The name Stoicism comes  from the Stoa Poikile, the decorated public colonnade even in the bleakest situations. to mean someone who  remains calm under pressure and avoids emotional extremes. But while this captures important aspects of Stoicism, the original philosophy was more than just an attitu

In [45]:
s=""
str=""
for key, value in res:
    #print(value)
    sst = s.join(value)
    str+=sst

print(str)

and screwed tightly shut. But the contents, Zeus told her,  were not for mortal eyes. She was not to open the box  under any circumstance. Monstrous creatures and horrendous sounds rushed out in a cloud of smoke and swirled around her, screeching and cackling. Filled with terror, Pandora clawed desperately at the air  to direct them back into their prison. Today, Pandoraâ€™s Box suggests  the extreme consequences of tampering with the unknown â€“ but Pandoraâ€™s burning curiosity also  suggests the duality that lies at the heart of human inquiry. Are we bound to investigate everything  we donâ€™t know, to mine the earth for more â€“ or are there some mysteries 


### 5. Intersection

### 6. Passing input

In [22]:
print (summarize(str,word_count=40))

As she wept, Pandora became aware of a sound echoing  from within the box.
She knew that opening the box  was irreversible â€“ but alongside the strife, sheâ€™d set hope forth to temper its effects.


In [23]:
print(summarize(lines_cleaned,word_count=40))

two gifts on Pandora.
On earth, Pandora met and fell in love 
Zeus had used the box as a vessel
Pandora became aware of a sound echoing 
Today, Pandoraâ€™s Box suggests 
to mine the earth for more â€“


In [24]:
sum_machine = summarize(lines_cleaned, word_count=40)
sum_workers = summarize(str, word_count=40)

### 7. Access preparation

In [25]:
api_key="juEydPi9AVH26V9AGTfv"

#Getting the response of the api call into a json file
response=requests.post("https://api.figure-eight.com/v1/jobs.json?key="+api_key)

In [26]:
#response status code
print(response.status_code)

200


In [27]:
#response content, job details
print(response.content)

b'{"id":1394638,"options":{"logical_aggregation":true,"mail_to":"w.huang-10@student.tudelft.nl","req_ttl_in_seconds":1800},"title":null,"secret":"w29Oq3myHcQXqsfiFCeNoiXay81l7wU+XKwK1XaHYjg6","project_number":null,"alias":null,"judgments_per_unit":3,"units_per_assignment":5,"pages_per_assignment":1,"max_judgments_per_worker":null,"gold_per_assignment":1,"minimum_account_age_seconds":null,"execution_mode":"worker_ui_remix","payment_cents":10,"design_verified":true,"public_data":false,"variable_judgments_mode":"none","max_judgments_per_unit":null,"expected_judgments_per_unit":null,"min_unit_confidence":null,"units_remain_finalized":null,"auto_order_timeout":null,"auto_order_threshold":4,"completed_at":null,"state":"unordered","auto_order":false,"webhook_uri":null,"send_judgments_webhook":null,"language":"en","minimum_requirements":null,"desired_requirements":null,"max_work_per_network":null,"copied_from":null,"created_at":"2019-06-16T14:50:22+00:00","updated_at":"2019-06-16T14:50:22+00:0

### 7. Create a task model here

In [178]:
#Creating task with title, instructions and no data
job_title = "Video validation task"
instructions="<h1>Overview</h1><p>Please play the video and select the most appropriate summary in terms of conclusiveness and accuracy of the summary. </p><hr><h1>Steps</h1><p>1. Play the video.</p><p>2. Understand the video content.</p><p>3. Inspect the two summary and understand them.</p><p>4. Select one summary that summarize the video better.</p>"
instructions+="<p>5. Submit your work.</p>"

In [179]:
#setup the request url
request_url = "https://api.figure-eight.com/v1/jobs.json"
headers = {'content-type': 'application/json'}

In [180]:
cml="""<div class="html-element-wrapper">
  <a target="_blank" href="{{link}}">
    <span>
      <img span="" class="fr-fic fr-dii" />
    </span>
    <span>{{link}}</span>
  </a>
  <p>
    <span>
      <iframe src="{{link}}" width="560" height="380" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture"></iframe>
    </span>
    <br />
  </p>
  <p>
    <br />
  </p>
</div>
<cml:radios validates="required" label="Please select the most appropriate summary from the candidates.">
  <cml:radio label="{{choice1}}" />
  <cml:radio label="{{choice2}}" />
</cml:radios>"""

In [181]:
#Setup the payload object
payload = {
'key': api_key,
'job':{
'title': job_title,
'instructions': instructions,
'cml': cml
}
}

response=requests.post(request_url, data=json.dumps(payload), headers=headers)
#print(response.content)

In [182]:
#find job id and save it
workJson = json.loads(response.content)
job_id = workJson['id']
print(job_id)

1394602


In [183]:
#Updating the task we had created before by adding data row by row (unit by unit)
API_KEY = "juEydPi9AVH26V9AGTfv"

data = {"link": "https://www.youtube.com/embed/pMdJxVjZMRI?cc_load_policy=1&color=white&loop=1&modestbranding=1", "choice1": sum_machine, "choice2": sum_workers}

request_url = "https://api.figure-eight.com/v1/jobs/{}/units.json".format(job_id)
headers = {'content-type': 'application/json'}

payload = {
'key': API_KEY,
'unit': {
'data': data
}
}
response=requests.post(request_url, data=json.dumps(payload), headers=headers)